In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split, Subset
import torchvision.transforms as transforms
import torchvision
import pickle
from torcheval.metrics import MulticlassAccuracy

In [2]:
device = torch.device('cuda')
device

device(type='cuda')

# Load Data

In [4]:
from torch.utils.data import Dataset


def binary(x, bits):
    mask = 2**torch.arange(bits).to(x.device, x.dtype)
    return x.unsqueeze(-1).bitwise_and(mask).ne(0).float()


class VariableLenDataset(Dataset):
    def __init__(self, data, targets):
        self.X = [torch.tensor(x, dtype=torch.float32) for x in data]
        self.y = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


class MusiciansDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, idx):
        image, label = self.data[idx]
        return image, label

    def __len__(self):
        return len(self.data)

In [5]:
with open("data/train.pkl", "rb") as train_file:
    dataset = pickle.load(train_file)

In [6]:
df = pd.DataFrame(dataset)
df

,0,1
0,"[-1.0, -1.0, -1.0, -1.0, 144.0, 144.0, 144.0, ...",0
1,"[-1.0, -1.0, 144.0, 144.0, 34.0, 0.0, 4.0, 2.0...",0
2,"[66.0, 100.0, 148.0, 148.0, 146.0, 64.0, 146.0...",0
3,"[147.0, 65.0, 64.0, 146.0, 8.0, 50.0, 160.0, 5...",0
4,"[144.0, 144.0, 144.0, 144.0, 144.0, 64.0, 64.0...",0
...,...,...
2934,"[144.0, 144.0, 144.0, 66.0, 82.0, 132.0, 8.0, ...",4
2935,"[32.0, 33.0, 162.0, 121.0, 68.0, 68.0, 120.0, ...",4
2936,"[33.0, 33.0, 33.0, 33.0, 78.0, 2.0, 68.0, 34.0...",4
2937,"[96.0, 96.0, 96.0, 96.0, 176.0, 176.0, 176.0, ...",4


In [6]:
train_dataset = VariableLenDataset(df[0].values, df[1].values)

In [7]:
base_dataset = VariableLenDataset(df[0].values, df[1].values)
torch.manual_seed(42)
n_train_examples = int(len(base_dataset) * 0.95)
train_indices, validation_indices = random_split(
    range(len(base_dataset)), [n_train_examples, len(base_dataset) - n_train_examples]
)

validation_dataset = MusiciansDataset(
    Subset(base_dataset, validation_indices)
)
train_dataset = MusiciansDataset(
    Subset(base_dataset, train_indices)
)

In [8]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

pad = 0

def pad_collate(batch, pad_value=0):
    xx, yy = zip(*batch)
    x_lens = [len(x) if x.dim() > 0 else 0 for x in xx]
    y_lens = [len(y) if y.dim() > 0 else 0 for y in yy]

    xx = [x.unsqueeze(0) if x.dim() == 0 else x for x in xx]
    yy = [y.unsqueeze(0) if y.dim() == 0 else y for y in yy]

    xx_pad = pad_sequence(xx, batch_first=False, padding_value=pad_value)
    yy_pad = pad_sequence(yy, batch_first=False, padding_value=pad_value)

    return xx_pad, yy_pad, x_lens, y_lens

In [9]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=pad_collate, num_workers=16)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=True, collate_fn=pad_collate, num_workers=16)

/home/mikic202/miniconda3/envs/pt/lib/python3.11/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [38]:
for a, b, c, d in train_loader:
    print(a)
    print(b)
    break

tensor([[ 60.,  -1.,  -1.,  ...,  -1.,   0., 112.],
        [ 20.,  -1.,  -1.,  ...,  -1., 145.,   0.],
        [172., 112.,  -1.,  ...,  -1.,  15.,  67.],
        ...,
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,  -1.,   0.,  ...,   0.,   0.,   0.]])
tensor([[0, 1, 1, 0, 0, 0, 1, 0, 2, 0, 0, 1, 4, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1,
         3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 0, 3, 0, 0, 1, 0, 4, 4, 0, 1, 0, 3, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 4, 0, 1, 1, 0, 1, 1, 0, 1]])


# Model

In [10]:
class ComposerClasifier(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, dropout=0.4)
        self.fc = nn.Linear(hidden_size, out_size)
        self.act =  nn.GELU()
        # self.fc2 = nn.Linear(32, out_size)
        # self.act2 =  nn.GELU()

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, x_lens, hidden):
        # x = torch.transpose(x,0,1)
        x_packed = pack_padded_sequence(x, x_lens, batch_first=False, enforce_sorted=False)
        all_outputs, hidden = self.lstm(x_packed, hidden)
        # preds, pred_len = pad_packed_sequence(all_outputs, batch_first=False, padding_value=0)
        # all_outputs = torch.transpose(all_outputs,0,1)
        # out = torch.flatten(all_outputs,1)
        # print(all_outputs.shape)
        # all_outputs = torch.transpose(all_outputs,0,1)
        x = self.act(self.fc(hidden[0][-1]))
        # x = self.act2(self.fc2(x))
        return x, hidden


In [21]:
# # GELU wagi [0.6, 1.5, 2.0, 1.6, 2.8] Accuracy 81% test
hidden_dim = 256
layer_dim = 3
input_dim = 1
out_dim = df[1].nunique()

# # SIgmoid wagi [0.5, 1.3, 2.0, 1.5, 2.3] dropout 0.3 Accuracy 5374% test
# hidden_dim = 128
# layer_dim = 4
# input_dim = 1
# out_dim = df[1].nunique()

# # GELU wagi [0.6, 1.4, 2.0, 1.6, 3.0] Accuracy 67% test
# hidden_dim = 64
# layer_dim = 4
# input_dim = 1
# out_dim = df[1].nunique()

# # GELU wagi [0.6, 1.4, 2.0, 1.6, 3.0] Accuracy 86% test
# hidden_dim = 164
# layer_dim = 4
# input_dim = 1
# out_dim = df[1].nunique()

# # GELU wagi [0.6, 1.4, 2.0, 1.7, 3.0] Accuracy 96% (na zbiorze treningowym)
# hidden_dim = 128
# layer_dim = 4
# input_dim = 1
# out_dim = df[1].nunique()

# # dodatkowa warstwa liniowa 55% (na zbiorze treningowym) same 0 możliwe że trzeba wagi zmienić
# hidden_dim = 128
# layer_dim = 4
# input_dim = 1
# out_dim = df[1].nunique()

# # Accuracy 96% (na zbiorze treningowym)
# hidden_dim = 128
# layer_dim = 4
# input_dim = 1
# out_dim = df[1].nunique()

# # Accuracy 95% (na zbiorze treningowym)
# hidden_dim = 128
# layer_dim = 6
# input_dim = 1
# out_dim = df[1].nunique()

# # Accuracy 46% (na zbiorze treningowym) wagi [0.6, 1.4, 2.0, 2.0, 3.2]
# hidden_dim = 128
# layer_dim = 8
# input_dim = 1
# out_dim = df[1].nunique()

model = ComposerClasifier(input_dim, hidden_dim, layer_dim, out_dim)
model.to(device)

ComposerClasifier(
  (lstm): LSTM(1, 256, num_layers=3, dropout=0.4)
  (fc): Linear(in_features=256, out_features=5, bias=True)
  (act): GELU(approximate='none')
)

In [26]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fun = nn.CrossEntropyLoss(torch.tensor([0.6, 1.7, 4.8, 1.6, 3.2]).to(device))

In [27]:
# Training loop
model.train()
for epoch in range(41):
    loss_sum = 0
    for x, targets, x_lens, target_lens in train_loader:
        x = x.to(device).unsqueeze(2)
        targets = torch.transpose(targets,0,1)
        targets = targets.to(device).squeeze(1)

        hidden, state = model.init_hidden(x.size(1))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = model(x, x_lens, (hidden,state))
        preds = preds.squeeze(1)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
    if epoch % 5 == 0:
        print(f"Epoch: {epoch}, loss: {loss_sum:.3}")

Epoch: 0, loss: 10.5
Epoch: 5, loss: 6.03
Epoch: 10, loss: 4.34
Epoch: 15, loss: 3.54
Epoch: 20, loss: 2.58
Epoch: 25, loss: 2.13


KeyboardInterrupt: 

In [12]:
model.eval()
total = 0
corect = 0
labels_train = []
labels_pred = []
mca_eval = MulticlassAccuracy(num_classes=5, average=None)
with torch.no_grad():
    for x, targets, x_len, target_len in train_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.shape[1])
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = model(x, x_len, (hidden, state))
        preds = torch.argmax(preds, dim=1)

        for pred, targ in zip(preds, targets[0]):
            total += 1
            if torch.equal(pred, targ):
                corect += 1
        labels_train.append(targets[0])
        labels_pred.append(preds)
        mask_tgt = targets != pad
        print(targets)
        print(preds)

print(corect/total)
mca_eval.update(torch.cat(labels_pred), torch.cat(labels_train))
print(mca_eval.compute())

tensor([[1, 1, 0, 2, 0, 1, 2, 1, 0, 0, 0, 4, 0, 0, 3, 4, 1, 0, 2, 0, 0, 0, 0, 0,
         0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 3, 0,
         0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 4, 3, 1, 0]], device='cuda:0')
tensor([1, 1, 0, 2, 0, 1, 2, 1, 0, 0, 0, 4, 0, 0, 3, 4, 1, 0, 2, 0, 0, 0, 0, 0,
        0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 3, 0,
        0, 4, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 4, 3, 1, 0], device='cuda:0')
tensor([[4, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 3, 0, 0, 1, 1, 3, 2, 2, 0, 0,
         0, 0, 0, 3, 2, 4, 1, 3, 0, 0, 0, 0, 0, 3, 1, 1, 3, 1, 0, 0, 3, 0, 3, 0,
         0, 3, 0, 1, 1, 4, 0, 3, 0, 3, 2, 0, 1, 0, 4, 1]], device='cuda:0')
tensor([4, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 3, 1, 3, 0, 3, 1, 1, 3, 2, 2, 0, 0,
        0, 0, 0, 3, 2, 4, 1, 3, 0, 0, 0, 0, 2, 3, 1, 1, 3, 1, 0, 0, 3, 0, 3, 0,
        0, 3, 0, 1, 1, 4, 0, 3, 0, 3, 2, 0, 1, 0, 4, 1], device='cuda:0')
tensor([[4, 0, 0, 1, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0,

/home/mikic202/miniconda3/envs/pt/lib/python3.11/site-packages/torcheval/metrics/functional/classification/accuracy.py:275: UserWarning: The reduce argument of torch.scatter with Tensor src is deprecated and will be removed in a future PyTorch release. Use torch.scatter_reduce instead for more reduction options. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:232.)
  num_correct = mask.new_zeros(num_classes).scatter_(0, target, mask, reduce="add")


In [19]:
torch.save(model, "model_test_86_39_słaby_podział.model")

# Fill Evaluate Data

In [46]:
model = torch.load("model_test_86_39_słaby_podział.model")

In [47]:
pad = 0

def pad_collate_test(batch, pad_value=0):
    x_lens = [len(x) if isinstance(x, torch.Tensor) and x.dim() > 0 else 0 for x in batch]
    xx = [x.unsqueeze(0) if isinstance(x, torch.Tensor) and x.dim() == 0 else x for x in batch]
    xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)
    return xx_pad, x_lens

In [48]:
with open("data/test_no_target.pkl", "rb") as test_file:
    test_data = pickle.load(test_file)


test_df = pd.DataFrame({'Sequence': test_data})

In [49]:
test_df

,Sequence
0,"[0.0, 0.0, 0.0, 0.0, 88.0, 88.0, 88.0, 88.0, 6..."
1,"[0.0, 0.0, 2.0, 18.0, 50.0, 82.0, 1.0, 1.0, 82..."
2,"[-1.0, 64.0, 64.0, 144.0, 144.0, 144.0, 3.0, 0..."
3,"[144.0, 146.0, 1.0, 132.0, 34.0, 100.0, 148.0,..."
4,"[96.0, 18.0, 100.0, 18.0, 36.0, 76.0, 50.0, 10..."
...,...
1098,"[112.0, 112.0, 0.0, 0.0, 0.0, 159.0, 190.0, 12..."
1099,"[112.0, 112.0, 112.0, 112.0, 124.0, 32.0, 32.0..."
1100,"[176.0, 176.0, 66.0, 146.0, 124.0, 190.0, 77.0..."
1101,"[32.0, 32.0, 32.0, 32.0, 33.0, 78.0, 92.0, 33...."


In [50]:
class TestVariableLenDataset(Dataset):
    def __init__(self, data):
        self.X = [torch.tensor(x, dtype=torch.float32) for x in data]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx]

In [51]:
test_dataset = TestVariableLenDataset(test_df["Sequence"].values)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=pad_collate_test, num_workers=16)

/home/mikic202/miniconda3/envs/pt/lib/python3.11/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [52]:
print(len(test_df["Sequence"].values))

1103


In [53]:

model.eval()
predicted_test = torch.tensor([], device=device)

with torch.no_grad():
    for x, x_lens in test_loader:
        x = torch.transpose(x, 0, 1)
        x = x.to(device).unsqueeze(2)

        hidden, state = model.init_hidden(x.size(1))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = model(x,x_lens, (hidden, state))
        preds = preds.squeeze(1)

        predicted_test = torch.cat((predicted_test, torch.argmax(preds, dim=1)))

In [54]:
print(predicted_test)
print(len(predicted_test))
print(torch.unique(predicted_test, return_counts=True))


tensor([3., 2., 0.,  ..., 4., 3., 4.], device='cuda:0')
1103
(tensor([0., 1., 2., 3., 4.], device='cuda:0'), tensor([608, 176,  55, 167,  97], device='cuda:0'))


In [55]:
pd.DataFrame(predicted_test.int().cpu().numpy()).to_csv("ChomanskiMłynik_output2.csv", index=False, header=None)